In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import healpy as hp
from mifipy import mfun
import seaborn as sns
import lmfit
from lmfit import Model
from lmfit.models import GaussianModel, ParabolicModel, ExpressionModel, ExponentialGaussianModel, MoffatModel, SkewedGaussianModel
from scipy import optimize

In [ ]:
def nearest_index_of(arrval, value):
    return np.argmin( abs(arrval - value) )

def gaussian_p(x, p):
    amp, mean, sigma = p
    return - amp / sigma**3 / np.sqrt(2 * np.pi) * (x - mean) * \
        np.exp(- (x - mean)**2. / 2 / sigma**2.)

def gaussian_pp(x, p):
    amp, mean, sigma = p
    return amp / sigma**5 / np.sqrt(2 * np.pi) * (x - mean) ** 2. * np.exp(- (x - mean)**2. / 2 / sigma**2.) \
        - amp / sigma**3 / np.sqrt(2 * np.pi) * np.exp(- (x - mean)**2. / 2 / sigma**2.) 

# An Ugly Function to Find 5 peaks beased on 1st derivative
def find_five_gaussians_roots(parameters):
    try:
        p1, p2, p3, p4, p5 = parameters.reshape(5, 3)
    except:
        p1, p2, p3, p4, p5 = np.array(parameters).reshape(5, 3)

    def five_gaussian_p(x):
        return sum([gaussian_p(x, p) for p in [p1, p2, p3, p4, p5]])
    
    def five_gaussian_pp(x):
        return sum([gaussian_pp(x, p) for p in [p1, p2, p3, p4, p5]])
    
    center_list = [p1[1], p2[1], p3[1], p4[1], p5[1]]
    opt_list = np.array(sorted([optimize.root(five_gaussian_p, i, jac=five_gaussian_pp).x[0] 
                                for i in center_list]))
    return [opt_list[nearest_index_of(opt_list, p[1])] for p in [p1, p2, p3, p4, p5]]

# update the values of peaks to replace gaussian centers
def updating_dataframe_peaks_with_roots(df):
    df_update = df.copy()
    peaks_string_list = ['peak1', 'peak2', 'peak3', 'peak4', 'peak5']
    
    # Best fit with 5 gaussians
    centers = pd.DataFrame([df_update.loc[:, pkstr] for pkstr in peaks_string_list]).T
    centers.columns = ['center1', 'center2', 'center3', 'center4', 'center5']

    for i in df_update.index:
        parameters = [df_update.loc[i, par + str(j)] for j in range(1,6) for par in ['amp', 'peak', 'sigma']]
        five_roots = find_five_gaussians_roots(parameters)
        for root, pkstr in zip(five_roots, peaks_string_list): 
            df_update.loc[i, pkstr] = root

    # concat df_update and centers
    return pd.concat([df_update, centers], axis=1)

In [ ]:
def updating_dictionary_peaks_with_roots(df):
    df_update = df.copy()
    peaks_string_list = ['g1_peak', 'g2_peak', 'g3_peak', 'g4_peak', 'g5_peak']
    parameters_list = ['g' + str(j) + '_' + par for j in range(1,6) 
                                                for par in ['amplitude', 'center', 'sigma']]
    peaks = {p : [] for p in peaks_string_list}
    for i in df_update.index:
        parameters = [df_update.loc[i, par] for par in parameters_list]
        five_roots = find_five_gaussians_roots(parameters)

        for root, pkstr in zip(five_roots, peaks_string_list): 
            peaks[pkstr].append(root)

    # concat df_update and centers
    return pd.concat([pd.DataFrame(peaks), df_update], axis=1)

In [ ]:
# gaussian_fit_helper updater
def update_gaussuan_helper_with_roots(result):
    parameters = [result.best_values[prefix + par] 
        for prefix in ['g1_', 'g2_', 'g3_', 'g4_', 'g5_']
        for par in ['amplitude', 'center', 'sigma']]
    peak_list = [prefix + 'peak' for prefix in ['g1_', 'g2_', 'g3_', 'g4_', 'g5_']]
    peaks = find_five_gaussians_roots(parameters)
    for name, peak in zip(peak_list, peaks):
        result.best_values[name] = peak
    return result

In [ ]:
def index_of(arrval, value):
    if value < min(arrval): return 0
    return max(np.where(arrval<=value)[0])

In [ ]:
def gaussians_fit_helper(ell, cl, return_fig=False, 
                         bound_peak=False, range_guess=False, method='leastsq'):
    '''fit power spectrum with gaussians
    parameter: 
    return_fig=True -> return figures for fitting result
    '''

    # find better data range 
    def index_of(arrval, value):
        if value < min(arrval): return 0
        return max(np.where(arrval<=value)[0])
    ix1 = index_of(ell,415.35107897)
    ix2 = index_of(ell,670.1145778)
    ix3 = index_of(ell,1020.37320212)
    ix4 = index_of(ell,1311.72424321)
    ix5 = index_of(ell,1604)
    
    # fits with gaussian functions 
    gauss1 = GaussianModel(prefix='g1_')
    gauss2 = GaussianModel(prefix='g2_')
    gauss3 = GaussianModel(prefix='g3_')
    gauss4 = GaussianModel(prefix='g4_')
    gauss5 = GaussianModel(prefix='g5_')
    
    # initial vaule 
    if range_guess==True:
        pars = gauss1.guess(cl[:ix1], x=ell[:ix1])
        pars += gauss2.guess(cl[ix1:ix2], x=ell[ix1:ix2])
        pars += gauss3.guess(cl[ix2:ix3], x=ell[ix2:ix3])
        pars += gauss4.guess(cl[ix3:ix4], x=ell[ix3:ix4])
        pars += gauss5.guess(cl[ix4:ix5], x=ell[ix4:ix5])
    if range_guess==False:
        pars = gauss1.guess(cl[:ix1], x=ell[:ix1])
        pars.update(gauss1.make_params())
        pars['g1_amplitude'].set(1383237.6375727942)
        pars['g1_center'].set(218.25413354426811) # 218.25413354426811
        pars['g1_sigma'].set(97.08110083797027)
        pars.update(gauss2.make_params())
        pars['g2_amplitude'].set(541007.6549703055)
        pars['g2_center'].set(532.7906394119341) # 532.7906394119341
        pars['g2_sigma'].set(86.81717396133978)
        pars.update(gauss3.make_params())
        pars['g3_amplitude'].set(692701.0866300496)
        pars['g3_center'].set(812.1818016015095) # 812.1818016015095
        pars['g3_sigma'].set(110.16506040901645)
        pars.update(gauss4.make_params())
        pars['g4_amplitude'].set(243336.67891973304)
        pars['g4_center'].set(1125.9129396789704) # 1125.9129396789704
        pars['g4_sigma'].set(92.67204801014215)
        pars.update(gauss5.make_params())
        pars['g5_amplitude'].set(382571.02244731307)
        pars['g5_center'].set(1424.7207136484815) # 1424.7207136484815
        pars['g5_sigma'].set(183.43056355023555)
    
    gmod =  gauss1 + gauss2 + gauss3 + gauss4 + gauss5
    
    # bound the parameter
    #pars['g1_sigma'].set(min=48.046687036192097, max=147.56693526257902)
    #pars['g2_sigma'].set(min=6.5892775323649744, max=183.75326766522033)
    #pars['g3_sigma'].set(min=35.193792564236674, max=183.98817016897786)

    if bound_peak==True:
        #pars['g1_center'].set(min=0, max=1700)
        #pars['g2_center'].set(min=0, max=1700)
        #pars['g3_center'].set(min=0, max=1700)
        #pars['g4_center'].set(min=0, max=1700)
        #pars['g5_center'].set(min=0, max=1700)
        pars['g1_sigma'].set(max=1000)
        pars['g2_sigma'].set(max=1000)
        pars['g3_sigma'].set(max=1000)
        pars['g4_sigma'].set(max=1000)
        pars['g5_sigma'].set(max=1000)
        pars['g1_amplitude'].set(min=0)
        pars['g2_amplitude'].set(min=0)
        pars['g3_amplitude'].set(min=0)
        pars['g4_amplitude'].set(min=0)
        pars['g5_amplitude'].set(min=0)
        
    # fit model to data array ecl 
    result = gmod.fit(cl, pars, x=ell, method=method) ## ell=ell[0:1600], ecl=ecl[0:1600]
    if return_fig==True:
        plt.plot(ell, cl, 'k', linestyle='None', marker='.')
        plt.plot(ell, result.best_fit, lw=2)

        # Components plots
        comps = result.eval_components(x=ell)
        plt.plot(ell, comps['g1_'])
        plt.plot(ell, comps['g2_'])
        plt.plot(ell, comps['g3_'])
        plt.plot(ell, comps['g4_'])
        plt.plot(ell, comps['g5_'])

        plt.xlabel(r'$\ell$')
        plt.ylabel(r'$\ell(\ell+1) C_\ell/2\pi$')
        plt.xlim([0., ell[-1]])

    return result

In [ ]:
def peak_df(ell, table_ecls, lmax, lmin, method='leastsq'):
    peak1 = []; peak2 = []; peak3 = []; peak4 = []; peak5 =[];
    sigma1 = []; sigma2 = []; sigma3 = []; sigma4 = []; sigma5 = [];
    amp1 = []; amp2 = []; amp3 = []; amp4 = []; amp5 = [];
    redchi  = [];
    for i in range(len(table_ecls)):
        result = gaussians_fit_helper(ell[index_of(ell, lmin):index_of(ell, lmax)], 
                                      table_ecls[i][index_of(ell, lmin):index_of(ell, lmax)],
                                      method=method)
        parms = result.params
        peak1.append(parms['g1_center'].value)
        peak2.append(parms['g2_center'].value)
        peak3.append(parms['g3_center'].value)
        peak4.append(parms['g4_center'].value)
        peak5.append(parms['g5_center'].value)
        sigma1.append(parms['g1_sigma'].value)
        sigma2.append(parms['g2_sigma'].value)
        sigma3.append(parms['g3_sigma'].value)
        sigma4.append(parms['g4_sigma'].value)
        sigma5.append(parms['g5_sigma'].value)
        amp1.append(parms['g1_amplitude'].value)
        amp2.append(parms['g2_amplitude'].value)
        amp3.append(parms['g3_amplitude'].value)
        amp4.append(parms['g4_amplitude'].value)
        amp5.append(parms['g5_amplitude'].value)
        
        redchi.append(result.redchi)
    d = {'peak1': peak1,
        'peak2' : peak2,
        'peak3' : peak3,
        'peak4' : peak4,
        'peak5' : peak5,
        'sigma1': sigma1,
        'sigma2': sigma2,
        'sigma3': sigma3,
        'sigma4': sigma4,
        'sigma5': sigma5,
        'amp1'  : amp1,
        'amp2'  : amp2,
        'amp3'  : amp3,
        'amp4'  : amp4,
        'amp5'  : amp5,
        'redchi': redchi}
    df = pd.DataFrame(d)
    del peak1, peak2, peak3, peak4, peak5, sigma1, sigma2, sigma3, sigma4, sigma5, redchi, parms, result
    
    # it is an ad-hoc function ... well I will fix the program next time
    df = updating_dataframe_peaks_with_roots(df)
    return df

In [ ]:
def outer_outlier_help(df_test, df_train, amp_cut=True, sigma_cut=True, sort_peak=False, 
                      first_three=True):
    # naive cutoff 3 std
    print('len of dataframe before cutoff :', len(df_test))
    
    if amp_cut==True:
        # amplitude cutoff
        df_test = df_test[df_test['amp1'] < (df_train['amp1'].quantile(q=0.75) + (df_train['amp1'].quantile(q=0.75) - df_train['amp1'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['amp1'] > (df_train['amp1'].quantile(q=0.25) - (df_train['amp1'].quantile(q=0.75) - df_train['amp1'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['amp2'] < (df_train['amp2'].quantile(q=0.75) + (df_train['amp2'].quantile(q=0.75) - df_train['amp2'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['amp2'] > (df_train['amp2'].quantile(q=0.25) - (df_train['amp2'].quantile(q=0.75) - df_train['amp2'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['amp3'] < (df_train['amp3'].quantile(q=0.75) + (df_train['amp3'].quantile(q=0.75) - df_train['amp3'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['amp3'] > (df_train['amp3'].quantile(q=0.25) - (df_train['amp3'].quantile(q=0.75) - df_train['amp3'].quantile(q=0.25)) * 3)]
        if first_three==False:
            df_test = df_test[df_test['amp4'] < (df_train['amp4'].quantile(q=0.75) + (df_train['amp4'].quantile(q=0.75) - df_train['amp4'].quantile(q=0.25)) * 3)]
            df_test = df_test[df_test['amp4'] > (df_train['amp4'].quantile(q=0.25) - (df_train['amp4'].quantile(q=0.75) - df_train['amp4'].quantile(q=0.25)) * 3)]
            df_test = df_test[df_test['amp5'] < (df_train['amp5'].quantile(q=0.75) + (df_train['amp5'].quantile(q=0.75) - df_train['amp5'].quantile(q=0.25)) * 3)]
            df_test = df_test[df_test['amp5'] > (df_train['amp5'].quantile(q=0.25) - (df_train['amp5'].quantile(q=0.75) - df_train['amp5'].quantile(q=0.25)) * 3)]
        print('len of dataframe after amplitude cutoff :', len(df_test))
    
    if sigma_cut==True:
        # sigma cutoff
        df_test = df_test[df_test['sigma1'] < (df_train['sigma1'].quantile(q=0.75) + (df_train['sigma1'].quantile(q=0.75) - df_train['sigma1'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['sigma1'] > (df_train['sigma1'].quantile(q=0.25) - (df_train['sigma1'].quantile(q=0.75) - df_train['sigma1'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['sigma2'] < (df_train['sigma2'].quantile(q=0.75) + (df_train['sigma2'].quantile(q=0.75) - df_train['sigma2'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['sigma2'] > (df_train['sigma2'].quantile(q=0.25) - (df_train['sigma2'].quantile(q=0.75) - df_train['sigma2'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['sigma3'] < (df_train['sigma3'].quantile(q=0.75) + (df_train['sigma3'].quantile(q=0.75) - df_train['sigma3'].quantile(q=0.25)) * 3)]
        df_test = df_test[df_test['sigma3'] > (df_train['sigma3'].quantile(q=0.25) - (df_train['sigma3'].quantile(q=0.75) - df_train['sigma3'].quantile(q=0.25)) * 3)]
        if first_three==False:
            df_test = df_test[df_test['sigma4'] < (df_train['sigma4'].quantile(q=0.75) + (df_train['sigma4'].quantile(q=0.75) - df_train['sigma4'].quantile(q=0.25)) * 3)]
            df_test = df_test[df_test['sigma4'] > (df_train['sigma4'].quantile(q=0.25) - (df_train['sigma4'].quantile(q=0.75) - df_train['sigma4'].quantile(q=0.25)) * 3)]
            df_test = df_test[df_test['sigma5'] < (df_train['sigma5'].quantile(q=0.75) + (df_train['sigma5'].quantile(q=0.75) - df_train['sigma5'].quantile(q=0.25)) * 3)]
            df_test = df_test[df_test['sigma5'] > (df_train['sigma5'].quantile(q=0.25) - (df_train['sigma5'].quantile(q=0.75) - df_train['sigma5'].quantile(q=0.25)) * 3)]
        print('len of dataframe after sigma cutoff :', len(df_test))
    
    if sort_peak==True:
        # sort alignment peaks
        df_test = df_test[df_test['peak1'] < df_test['peak2']]
        df_test = df_test[df_test['peak1'] < df_test['peak3']]
        df_test = df_test[df_test['peak2'] < df_test['peak3']]
        if first_three==False:
            df_test = df_test[df_test['peak1'] < df_test['peak4']]
            df_test = df_test[df_test['peak1'] < df_test['peak5']]
            df_test = df_test[df_test['peak2'] < df_test['peak4']]
            df_test = df_test[df_test['peak2'] < df_test['peak5']]
            df_test = df_test[df_test['peak3'] < df_test['peak4']]
            df_test = df_test[df_test['peak3'] < df_test['peak5']]
            df_test = df_test[df_test['peak4'] < df_test['peak5']]
        print('len of dataframe after sorting alignment peaks :', len(df_test))

    return df_test

In [ ]:
def bad_df(df, first_three=False):
    # pandas index to list
    pre_idx = df.index.tolist()

    # outer outlier eliminating
    df_cut = outer_outlier_help(df, df, amp_cut=True, sigma_cut=True, sort_peak=True, first_three=first_three)
    aft_idx = df_cut.index.tolist()

    # fancy indexing with boolean indicing operator !=
    boolean_list = [idx not in aft_idx for idx in pre_idx]
    df_bad = df[boolean_list]
    return df_bad

In [ ]:
def cart_healpix(cartview, nside):
    '''read in an matrix and return a healpix pixelization map'''
    # Generate a flat Healpix map and angular to pixels
    healpix = np.zeros(hp.nside2npix(nside), dtype=np.double)
    hptheta = np.linspace(0, np.pi, num=cartview.shape[0])[:, None]
    hpphi = np.linspace(-np.pi, np.pi, num=cartview.shape[1])
    pix = hp.ang2pix(nside, hptheta, hpphi)
    
    # re-pixelize
    healpix[pix] = np.fliplr(np.flipud(cartview))
    
    return healpix

In [ ]:
def healpix_rotate(healpix_map, rot):
    # pix-vec
    ipix = np.arange(len(healpix_map))
    nside = np.sqrt(len(healpix_map) / 12)
    if int(nside) != nside: return print('invalid nside');
    nside = int(nside)
    vec = hp.pix2vec(int(nside), ipix)
    rot_vec = (hp.rotator.Rotator(rot=rot)).I(vec)
    irotpix = hp.vec2pix(nside, rot_vec[0], rot_vec[1], rot_vec[2])
    return np.copy(healpix_map[irotpix])

In [ ]:
def lonlat_block_shorten(rotate, blocksize=(18, 18), xsize=1080, nside=64):
    lon, lat, psi = rotate
    cartview = np.zeros((xsize/2, xsize))
    
    # center block
    blocksize = (blocksize[0]/360*xsize, blocksize[0]/360*xsize)
    cartview[xsize/4 - blocksize[1]/2: xsize/4 + blocksize[1]/2, 
             xsize/2 - blocksize[0]/2: xsize/2 + blocksize[0]/2] = np.ones((blocksize[0], blocksize[1]))
    
    # re-pixelization
    healpix = cart_healpix(cartview, nside)
    del cartview

    # cartview back to origin: theta 
    healpix = healpix_rotate(healpix, (0,-lat))
    
    return cart_healpix(cartview, nside)

A cute function to calculate the shift and variance of peak positions compared to bestfit $\Lambda$CDM

In [ ]:
def shift_df_concat(df, return_abs=False):
    # Call in Best-Fit
    best = np.loadtxt('../Release2/COM_PowerSpect_CMB-base-plikHM-TT-lowTEB-minimum-theory_R2.02.txt')
    L = best.T[0]
    CL = best.T[1]
    C1 = ['peak1', 'peak2', 'peak3']
    C2 = ['g1_peak', 'g2_peak', 'g3_peak']
    
    # Best fit with 5 gaussians
    best = gaussians_fit_helper(L[0:1604], CL[0:1604], return_fig=False)
    best = update_gaussuan_helper_with_roots(best)
    shift = pd.DataFrame([df.loc[:, c1] - best.best_values[c2] for c1, c2 in zip(C1, C2)]).T
    shift.columns = ['shift1','shift2','shift3']

    # concat the total shift
    if return_abs==True:
        shift = pd.concat([shift, np.sum(abs(shift), axis=1), np.sqrt(np.sum(shift**2, axis=1))], axis=1)
    else:
        shift = pd.concat([shift, np.sum(shift, axis=1), np.sqrt(np.sum(shift**2, axis=1))], axis=1)
    
    shift.columns = ['shift1','shift2','shift3', 'shift_sum', 'shift_var_distance']
    
    # concat df and shift
    return pd.concat([df, shift], axis=1)

In [ ]:
def self_df_concat(df, return_lensed=False, step=None, stop=None):
    '''
    Concentrate a given dataframe with its ['peak1':'peak3] subutracted with its mean values. 
    Users may be able to selected a divided range with slices keyword argument. 
    For a dataframe with length 140, slices=70 impies subutract 0~69 with 0~69 mean values, 70~139 with 70~139 mean values.
    Parameters:
    ----------
    df : dataframe.
    return_lensed : boolean.
    slices : int
    Returns:
    df : dataframe
    -------
    '''
    C1 = ['peak1', 'peak2', 'peak3']
    
    if return_lensed==True:
        temp = [pd.DataFrame([df.loc[i:i + step - 1, c1] - df.loc[i:i + step - 1, c1].mean() 
                       for c1 in C1]).T 
                for i in range(0, stop, step)]
        shift = pd.concat(temp, axis=0)
        shift.columns = ['shift1','shift2','shift3']
    if return_lensed==False:
        shift = pd.DataFrame([df.loc[:, c1] - df.loc[:, c1].mean() for c1 in C1]).T
        shift.columns = ['shift1','shift2','shift3']

    # concat the total shift
    shift = pd.concat([shift, np.sum(shift, axis=1), np.sum(abs(shift), axis=1), np.sqrt(np.sum(shift**2, axis=1))], axis=1)
    shift.columns = ['shift1','shift2','shift3', 'shift_sum', 'shift_abs_sum', 'shift_var_distance']
    
    # concat df and shift
    return pd.concat([df, shift], axis=1)

In [ ]:
def weighted_sky_map(df, a, b, size, theta_size, nside, patch_size=240):
    IMP = np.zeros(hp.nside2npix(nside), dtype=np.double)

    # suboptimal choice: let bad-fit become zero
    df_bad = bad_df(df)
    bad_index = list(df_bad.index)

    # subpotimal choice: rebulid list without bad-fit
    shift_array = np.copy(df['shift_var_distance'].values)
    if len(bad_index) != 0:
        shift_array[bad_index] = 0
    importance_list = list_rebuild(shift_array, b)

    for (i,theta),importance in zip(enumerate(a), importance_list):
        print(importance)
        IMP += Yakitori.Shift_Block_Rotater(theta * theta_size, size, nside, 
                                            b[i], importance, patch_size=patch_size)
    return IMP



In [ ]:
def list_rebuild(index, b):
    '''
    index: list, the list you want to rebuild
    b: list, the base list you want to take it as template to rebuid a list
    return:
    rebuild_list: list, reconstructed list based on b.
    '''
    # generate aggregate index
    length = [len(B) for B in b]
    if len(index) == np.sum(length):
        sum_length = np.insert(np.add.accumulate(length), 0, 0) # accumulate is cool
        return [index[sum_length[i]:sum_length[i+1]] for i,l in enumerate(sum_length[:-1])]
    else: print ('Not the same length!')

In [ ]:
def ones_circle(length):
    circle = np.zeros((length, length))
    y, x = np.indices((circle.shape))
    center = np.array([(x.max() - x.min()) / 2.0, (x.max() - x.min()) / 2.0])
    r = np.sqrt((x - center[0]) ** 2 + (y - center[1]) ** 2)
    circle[r < length / 2] = 1
    return circle

In [1]:
def ones_number_circle(length, number):

    from PIL import Image, ImageDraw, ImageFont
    from matplotlib.image import pil_to_array
    
    # decide your font
    fnt = ImageFont.truetype('/Library/Fonts/Georgia.ttf', int(length * 0.626)) # 135
    circle = ones_circle(length)
    txt = Image.new('L', size=(length, length), color=1)
    draw = ImageDraw.Draw(txt)
    
    w, h = draw.textsize(number, font=fnt)
    
    draw.text(((length - w) / 2, (length - h) / 4), text=number, font=fnt, align='center')
    txt = pil_to_array(txt)
    return circle * txt

# Legacy: one time functions

In [1]:
def gaussians_fit_bounds(ell, cl, bounds,
                         return_fig=False):
    '''fit power spectrum with gaussians
    parameter: 
    bounds : bounds in 2darray, [[sigma1_min, sigma1_max],[sigma2_min, sigma2_max], ...]
    return_fig=True : return figures for fitting result
    '''

    # find better data range 
    def index_of(arrval, value):
        if value < min(arrval): return 0
        return max(np.where(arrval<=value)[0])
    ix1 = index_of(ell,400)
    ix2 = index_of(ell,600)
    ix3 = index_of(ell,1000)
    ix4 = index_of(ell,1300)
    ix5 = index_of(ell,1600)

    # fits with gaussian functions 
    gauss1 = GaussianModel(prefix='g1_')
    gauss2 = GaussianModel(prefix='g2_')
    gauss3 = GaussianModel(prefix='g3_')
    gauss4 = GaussianModel(prefix='g4_')
    gauss5 = GaussianModel(prefix='g5_')
    
    gmod =  gauss1 + gauss2 + gauss3 + gauss4 + gauss5

    # initial vaule 
    pars = gauss1.guess(cl[:ix1], x=ell[:ix1])
    pars += gauss2.guess(cl[ix1:ix2], x=ell[ix1:ix2])
    pars += gauss3.guess(cl[ix2:ix3], x=ell[ix2:ix3])
    pars += gauss4.guess(cl[ix3:ix4], x=ell[ix3:ix4])
    pars += gauss5.guess(cl[ix4:ix5], x=ell[ix4:ix5])
    
    pars['g1_sigma'].set(min=bounds[0][0], max=bounds[0][1])
    pars['g2_sigma'].set(min=bounds[1][0], max=bounds[1][1])
    pars['g3_sigma'].set(min=bounds[2][0], max=bounds[2][1])
    pars['g4_sigma'].set(min=bounds[3][0], max=bounds[3][1])
    pars['g5_sigma'].set(min=bounds[4][0], max=bounds[4][1])
    pars['g1_amplitude'].set(min=0)
    pars['g2_amplitude'].set(min=0)
    pars['g3_amplitude'].set(min=0)
    pars['g4_amplitude'].set(min=0)
    pars['g5_amplitude'].set(min=0)

    # fit model to data array ecl 
    result = gmod.fit(cl, pars, x=ell) ## ell=ell[0:1600], ecl=ecl[0:1600]
    if return_fig==True:
        plt.plot(ell, cl, 'k', linestyle='None', marker='.')
        plt.plot(ell, result.best_fit, lw=2)

        # Components plots
        comps = result.eval_components(x=ell)
        plt.plot(ell, comps['g1_'])
        plt.plot(ell, comps['g2_'])
        plt.plot(ell, comps['g3_'])
        plt.plot(ell, comps['g4_'])
        plt.plot(ell, comps['g5_'])

        plt.xlabel(r'$\ell$')
        plt.ylabel(r'$\ell(\ell+1) C_\ell/2\pi$')
        plt.xlim([0., ell[-1]])

    return result

In [ ]:
def peak_df_bounds(ell, table_ecls, bounds):
    peak1 = []; peak2 = []; peak3 = []; peak4 = []; peak5 =[];
    sigma1 = []; sigma2 = []; sigma3 = []; sigma4 = []; sigma5 = [];
    amp1 = []; amp2 = []; amp3 = []; amp4 = []; amp5 = [];
    redchi  = [];
    for i in range(len(table_ecls)):
        result = gaussians_fit_bounds(ell[:index_of(ell, 1700)], 
                                     cls[i][:index_of(ell, 1700)],
                                     bounds)
        parms = result.params
        peak1.append(parms['g1_center'].value)
        peak2.append(parms['g2_center'].value)
        peak3.append(parms['g3_center'].value)
        peak4.append(parms['g4_center'].value)
        peak5.append(parms['g5_center'].value)
        sigma1.append(parms['g1_sigma'].value)
        sigma2.append(parms['g2_sigma'].value)
        sigma3.append(parms['g3_sigma'].value)
        sigma4.append(parms['g4_sigma'].value)
        sigma5.append(parms['g5_sigma'].value)
        amp1.append(parms['g1_amplitude'].value)
        amp2.append(parms['g2_amplitude'].value)
        amp3.append(parms['g3_amplitude'].value)
        amp4.append(parms['g4_amplitude'].value)
        amp5.append(parms['g5_amplitude'].value)
        
        redchi.append(result.redchi)
    d = {'peak1': peak1,
        'peak2' : peak2,
        'peak3' : peak3,
        'peak4' : peak4,
        'peak5' : peak5,
        'sigma1': sigma1,
        'sigma2': sigma2,
        'sigma3': sigma3,
        'sigma4': sigma4,
        'sigma5': sigma5,
        'amp1'  : amp1,
        'amp2'  : amp2,
        'amp3'  : amp3,
        'amp4'  : amp4,
        'amp5'  : amp5,
        'redchi': redchi}
    df = pd.DataFrame(d)
    del peak1, peak2, peak3, peak4, peak5, sigma1, sigma2, sigma3, sigma4, sigma5, redchi, parms, result
    return df

In [ ]:
def MLplot_help(y_test, y_predict):
    '''
    plot all columns in y_test compare with predict results (e.g. DecisionTree.predict, Knn.predict)
    Parameter:
    y_test: test set from test_train_split
    y_predict: prediction result from x_test, trained by (x_train, y_train)
    '''
    for i, column in enumerate(y_test.columns):
        b = np.c_[y_test[column].values, y_predict[:,i]]

        plt.figure(1)
        g = sns.JointGrid(b[:,0],b[:,1])
        g.plot_marginals(sns.distplot, color=".5")
        g.plot_joint(plt.hexbin, bins='log', gridsize=30, cmap=cold_cmap, extent=[0, np.max(b[:,0]), 0, np.max(b[:,0])])
        a = np.linspace(min(b[:,0]),max(b[:,0]),20)
        plt.plot(a,a,'k--')

        plt.title(column)
        plt.xlabel('$\sigma _{test}$', fontsize=16)
        plt.ylabel('$\sigma _{predict}$', fontsize=16)
        plt.xlim([min(b[:,0]),np.max(b[:,0])])
        plt.ylim([min(b[:,0]),np.max(b[:,0])])

        cax = g.fig.add_axes([1, 0.20, .01, 0.5])
        cb = plt.colorbar(cax=cax)
        cb.set_label('$\log_{10}(\mathcal{N})$')

In [ ]:
def gaussians_fit_train(ell, cl, center, sig_pred, return_fig=False):
    '''fit power spectrum with gaussians
    parameter: 
    center: numpy array 5 length
    sig_pred: numpy array 5 length
    return_fig=True -> return figures for fitting result
    '''
    a = 1700/2/np.sum(sig_pred)
    ix1a, ix1b = index_of(ell, center[0] - a*sig_pred[0]), index_of(ell, center[0] + a*sig_pred[0])
    ix2a, ix2b = index_of(ell, center[1] - a*sig_pred[1]), index_of(ell, center[1] + a*sig_pred[1])
    ix3a, ix3b = index_of(ell, center[2] - a*sig_pred[2]), index_of(ell, center[2] + a*sig_pred[2])
    ix4a, ix4b = index_of(ell, center[3] - a*sig_pred[3]), index_of(ell, center[3] + a*sig_pred[3])
    ix5a, ix5b = index_of(ell, center[4] - a*sig_pred[4]), index_of(ell, center[4] + a*sig_pred[4])

    
    # fits with gaussian functions 
    gauss1 = GaussianModel(prefix='g1_')
    gauss2 = GaussianModel(prefix='g2_')
    gauss3 = GaussianModel(prefix='g3_')
    gauss4 = GaussianModel(prefix='g4_')
    gauss5 = GaussianModel(prefix='g5_')
    gmod =  gauss1 + gauss2 + gauss3 + gauss4 + gauss5

    # initial vaule 
    try:
        pars = gauss1.guess(cl[ix1a:ix1b], x=ell[ix1a:ix1b])
        pars += gauss2.guess(cl[ix2a:ix2b], x=ell[ix2a:ix2b])
        pars += gauss3.guess(cl[ix3a:ix3b], x=ell[ix3a:ix3b])
        pars += gauss4.guess(cl[ix4a:ix4b], x=ell[ix4a:ix4b])
        pars += gauss5.guess(cl[ix5a:ix5b], x=ell[ix5a:ix5b])
    except:
        print([ix1a, ix1b, ix2a, ix2b, ix3a, ix3b, ix4a, ix4b, ix5a, ix5b])
        pars = gauss1.guess(cl[ix1a:ix1b], x=ell[ix1a:ix1b])
        pars += gauss2.guess(cl[ix1b:ix2b], x=ell[ix1b:ix2b])
        pars += gauss3.guess(cl[ix2b:ix3b], x=ell[ix2b:ix3b])
        pars += gauss4.guess(cl[ix3b:ix4b], x=ell[ix3b:ix4b])
        pars += gauss5.guess(cl[ix4b:ix5b], x=ell[ix4b:ix5b])
        print('->', [ix1a, ix1b, ix1b, ix2b, ix2b, ix3b, ix3b, ix4b, ix4b, ix5b])
        
    # bound the parameter
    pars['g1_center'].set(min=0)
    pars['g2_center'].set(min=0)
    pars['g3_center'].set(min=0)
    pars['g4_center'].set(min=0)
    pars['g5_center'].set(min=0)
    pars['g1_sigma'].set(min=0)
    pars['g2_sigma'].set(min=0)
    pars['g3_sigma'].set(min=0)
    pars['g4_sigma'].set(min=0)
    pars['g5_sigma'].set(min=0)
    pars['g1_amplitude'].set(min=0)
    pars['g2_amplitude'].set(min=0)
    pars['g3_amplitude'].set(min=0)
    pars['g4_amplitude'].set(min=0)
    pars['g5_amplitude'].set(min=0)
        
    # fit model to data array ecl 
    result = gmod.fit(cl, pars, x=ell) ## ell=ell[0:1600], ecl=ecl[0:1600]
    if return_fig==True:
        plt.plot(ell, cl, 'k', linestyle='None', marker='.')
        plt.plot(ell, result.best_fit, lw=2)

        # Components plots
        comps = result.eval_components(x=ell)
        plt.plot(ell, comps['g1_'])
        plt.plot(ell, comps['g2_'])
        plt.plot(ell, comps['g3_'])
        plt.plot(ell, comps['g4_'])
        plt.plot(ell, comps['g5_'])

        plt.xlabel(r'$\ell$')
        plt.ylabel(r'$\ell(\ell+1) C_\ell/2\pi$')
        plt.xlim([0., ell[-1]])

    return result

In [ ]:
def peak_df_train(ell, table_ecls, data, Regressor, bound_peak=False):
    '''
    input ell and cls with dataframe of previous run, plus ML regressor fit, 
    expect having good initial value prediction.
    you should run simulations to get the G(center-> sigma) mapping relation,
    and use 
    '''
    peak1 = []; peak2 = []; peak3 = []; peak4 = []; peak5 =[];
    sigma1 = []; sigma2 = []; sigma3 = []; sigma4 = []; sigma5 = [];
    amp1 = []; amp2 = []; amp3 = []; amp4 = []; amp5 = [];
    redchi  = [];
    
    # ML prediction based on center positions
    center = data.loc[:,'peak1':'peak5'].values
    center = np.array([sorted(T) for T in center]) ## sorting gaussian centers prevent bad prediction
    sig_pred = Regressor.predict(center)  ## ML prediction

    for i in range(len(table_ecls)):
        if bound_peak==True:
            result = gaussians_fit_helper(ell[:index_of(ell, 1700)], 
                                         cls[i][:index_of(ell, 1700)],
                                         bound_peak=bound_peak)
        else:
            result = gaussians_fit_train(ell[:index_of(ell, 1700)], 
                                         cls[i][:index_of(ell, 1700)],
                                         center[i], sig_pred[i])
        parms = result.params
        peak1.append(parms['g1_center'].value)
        peak2.append(parms['g2_center'].value)
        peak3.append(parms['g3_center'].value)
        peak4.append(parms['g4_center'].value)
        peak5.append(parms['g5_center'].value)
        sigma1.append(parms['g1_sigma'].value)
        sigma2.append(parms['g2_sigma'].value)
        sigma3.append(parms['g3_sigma'].value)
        sigma4.append(parms['g4_sigma'].value)
        sigma5.append(parms['g5_sigma'].value)
        amp1.append(parms['g1_amplitude'].value)
        amp2.append(parms['g2_amplitude'].value)
        amp3.append(parms['g3_amplitude'].value)
        amp4.append(parms['g4_amplitude'].value)
        amp5.append(parms['g5_amplitude'].value)
        
        redchi.append(result.redchi)
    d = {'peak1': peak1,
        'peak2' : peak2,
        'peak3' : peak3,
        'peak4' : peak4,
        'peak5' : peak5,
        'sigma1': sigma1,
        'sigma2': sigma2,
        'sigma3': sigma3,
        'sigma4': sigma4,
        'sigma5': sigma5,
        'amp1'  : amp1,
        'amp2'  : amp2,
        'amp3'  : amp3,
        'amp4'  : amp4,
        'amp5'  : amp5,
        'redchi': redchi}
    df = pd.DataFrame(d)
    del peak1, peak2, peak3, peak4, peak5, sigma1, sigma2, sigma3, sigma4, sigma5, redchi, parms, result
    return df

In [ ]:
# ordinary scipy optimize fitting
lmin, lmax, lbinwidth = 0. , 1700., 20. # not sure the value

# 5 mixture gaussian models
def model(x, norm1, mean1, sigma1, 
          norm2, mean2, sigma2, 
          norm3, mean3, sigma3, 
          norm4, mean4, sigma4, 
          norm5, mean5, sigma5):
    
    gaussian1 = norm1*lbinwidth/(2.*np.pi)**0.5/sigma1 * np.exp(-0.5*((x-mean1)/sigma1)**2)
    gaussian2 = norm2*lbinwidth/(2.*np.pi)**0.5/sigma2 * np.exp(-0.5*((x-mean2)/sigma2)**2)
    gaussian3 = norm3*lbinwidth/(2.*np.pi)**0.5/sigma3 * np.exp(-0.5*((x-mean3)/sigma3)**2)
    gaussian4 = norm4*lbinwidth/(2.*np.pi)**0.5/sigma4 * np.exp(-0.5*((x-mean4)/sigma4)**2)
    gaussian5 = norm5*lbinwidth/(2.*np.pi)**0.5/sigma5 * np.exp(-0.5*((x-mean5)/sigma5)**2)
    return gaussian1 + gaussian2 + gaussian3 + gaussian4 + gaussian5

In [ ]:
# cost function with 2 chosen variable 
def cost_x_y(x, y, position, ell, cl, p):
    '''keep only 2 variables for cost function. you are able to select which you want to keep by position=(px,py)'''
    p[position[0]] = x
    p[position[1]] = y
    expected = model(ell,p[0],p[1],p[2],
                     p[3],p[4],p[5],
                     p[6],p[7],p[8],
                     p[9],p[10],p[11],
                     p[12],p[13],p[14])
    
    clerr = cl**0.5
    delta = (cl[cl>0] - expected[cl>0])/clerr[cl>0]
    return (delta**2).sum()

def pars_contour(position, ell, cl, amp, center, sigma, p):
    label = ['amp', 'center', 'sigma']
    X = [amp, center, sigma][position[0]%3]
    Y = [amp, center, sigma][position[1]%3]
    XV, YV = np.meshgrid(X, Y)
    img = np.array([cost_x_y(x, y, position=position, 
        ell=ell, cl=cl, p=p) for x, y in zip(XV.ravel(), YV.ravel())]).reshape(size, size)
    
    # img plot
    plt.figure()
    plt.imshow(np.flipud(img), cmap='terrain', aspect='auto', extent=[X.min(), X.max(), Y.min(), Y.max()]); 
    plt.colorbar();
    plt.xlabel(label[position[0]%3] + str(position[0]//3 + 1))
    plt.ylabel(label[position[1]%3] + str(position[1]//3 + 1))
    plt.tight_layout()
    
    # contour plot
    plt.figure()
    plt.contourf(XV, YV, img, 35, alpha=.75, cmap=cold_cmap)
    plt.colorbar()
    ctr = plt.contour(XV, YV, img, 35, colors='gray', linewidth=.1, linestyles='dotted')
    plt.clabel(ctr, fontsize=8)
    plt.xlabel(label[position[0]%3] + str(position[0]//3 + 1))
    plt.ylabel(label[position[1]%3] + str(position[1]//3 + 1))
    plt.tight_layout()
    del img, X, Y, XV, YV

In [ ]:
import lmfit
from lmfit.models import PowerLawModel, LinearModel

def NPB_fit(phi_size, cla_path, clb_path, start_ell=1080,
           return_result=False, return_figure=True, return_linear=False, compare_best=False):
    '''ClB-ClA, and fit this curve with power law'''
    
    # Load in cla and ell array
    if compare_best==False: clA = np.loadtxt(cla_path)
    clB = np.loadtxt(clb_path)
    ell = np.arange(0,2500,360//phi_size)
    
    # Load in best-fit
    if compare_best==True: 
        try: best = np.loadtxt(cla_path)
        except: print('Please use cla as best-fit input path :)')
        L = best.T[0]
        clA = (best.T[1] / L / (L + 1) * 2 * np.pi)
        
        # scalling 
        clA = clA[0:2500:360//phi_size][0:len(clB)]

    # Get the mean value
    clA_mean = np.sum(clA, axis=0)/len(clA)
    clB_mean = np.sum(clB, axis=0)/len(clB)

    # Difference
    Difference = clB_mean - clA_mean
    Difference = Difference[index_of(ell, start_ell):index_of(ell, 2500)] # elimnate singularity
    ell = ell[index_of(ell, start_ell):index_of(ell, 2500)]

    ## PowerLaw
    power1 = PowerLawModel(prefix='p1_')
    pars = power1.guess(Difference, x=ell)
    result = power1.fit(Difference, pars, x=ell)
    if return_figure==True:
        plt.figure()
        plt.loglog(ell,Difference,'k',ls='',marker='.')
        plt.loglog(ell, result.best_fit,lw=2)
        plt.xlabel(r'$\ell$')
        plt.ylabel(r'$C_\ell^B - C_\ell^A \times b_{\ell}^2$')
        print(result.fit_report())
    
    if return_linear==True:
        ## Linear
        ell = np.log10(ell)
        Difference = np.log10(Difference)
        lin1 = LinearModel(prefix='l1_')
        pars = lin1.guess(Difference, x=ell)
        l1_result = lin1.fit(Difference, pars, x=ell)

        if return_figure==True:
            plt.figure()
            plt.plot(ell, Difference,'k',ls='',marker='.')
            plt.plot(ell, l1_result.best_fit,lw=2)
            plt.xlabel(r'$\ell$')
            plt.ylabel(r'$C_\ell^B - C_\ell^A \times b_{\ell}^2$')
            print(l1_result.fit_report())
        return l1_result
    else:
        return result

In [ ]:
def scipy_gaussian_helper(ell, cl, method, maxiter=None):
    # test with one of the cls
    cl = cl[0: index_of(ell, 1604)]
    ell = ell[0: index_of(ell, 1604)]
    clerr = cl**0.5

    # ordinary scipy optimize fitting
    lmin, lmax, lbinwidth = 0. , 1604., ell[1] - ell[0] # not sure the value

    # gausian function
    def gaussian(x, norm, mean, sigma):
        return norm/(2.*np.pi)**0.5/sigma * np.exp(-0.5*((x - mean)/sigma)**2)

    # 5 mixture gaussian models
    def model(x, norm1, mean1, sigma1, 
              norm2, mean2, sigma2, 
              norm3, mean3, sigma3, 
              norm4, mean4, sigma4, 
              norm5, mean5, sigma5):

        gaussian1 = gaussian(x, norm1, mean1, sigma1)
        gaussian2 = gaussian(x, norm2, mean2, sigma2)
        gaussian3 = gaussian(x, norm3, mean3, sigma3)
        gaussian4 = gaussian(x, norm4, mean4, sigma4)
        gaussian5 = gaussian(x, norm5, mean5, sigma5)
        return gaussian1 + gaussian2 + gaussian3 + gaussian4 + gaussian5

    # Cost function 
    def cost(p):
        expected = model(ell,p[0],p[1],p[2],
                         p[3],p[4],p[5],
                         p[6],p[7],p[8],
                         p[9],p[10],p[11],
                         p[12],p[13],p[14])

        delta = (cl[cl>0] - expected[cl>0])/clerr[cl>0]
        return (delta**2).sum()

    # setting initial values
    p_init = np.array([1401655.7262623503, 218.25413354426811, 98.93376062262756,
                       523354.98339826055, 532.7906394119341, 84.4019228730886,
                       707106.4789739547, 812.1818016015095, 112.37624448976163,
                       252763.19470124107, 1125.9129396789704, 91.72842845856799,
                       291150.7466517695, 1424.7207136484815, 146.01454002758945])
    r = opt.minimize(cost, p_init, method=method, options={'maxiter': maxiter}) #L-BFGS-B, Powell, SLSQP,
    
    if r.success:
        cx = np.linspace(lmin, lmax, 1604.)
        cy = model(cx,r.x[0],r.x[1],r.x[2],
                   r.x[3],r.x[4],r.x[5],
                   r.x[6],r.x[7],r.x[8],
                   r.x[9],r.x[10],r.x[11],
                   r.x[12],r.x[13],r.x[14])
        plt.plot(ell, cl, ls='', marker='.', color='black')
        plt.plot(cx, cy)

        # component plots
        plt.plot(cx, gaussian(cx, r.x[0], r.x[1], r.x[2]))
        plt.plot(cx, gaussian(cx, r.x[3], r.x[4], r.x[5]))
        plt.plot(cx, gaussian(cx, r.x[6], r.x[7], r.x[8]))
        plt.plot(cx, gaussian(cx, r.x[9], r.x[10], r.x[11]))    
        plt.plot(cx, gaussian(cx, r.x[12], r.x[13], r.x[14]))
        
        return r
    else:
        print(r.message)